In [ ]:
import sklearn, warnings, contextlib, IPython, math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Assignment 2 - Missing Values

This notebook is built on the notebook submitted for assignment one.

### Utility

This is a small utility for printing progress - there are some long operations later one and it is handy to have an indicator.
<br>(It does slow down the loop though, which is annoying)

In [ ]:
from IPython.display import display, update_display, Markdown

class Progress:
    def __init__(self, name, count):
        self.name = name
        self.count = count
        self._display("Starting '" + name + "'")
        self.last_update = 0
        self.step = math.ceil(self.count/100) # Update when about 1% has elapsed.
    
    def update(self, value):
        if value != self.count and int(value - self.last_update) % self.step > 0:
            return # Don't update too often since that will slow down loops a lot!
        self.last_update = value
        p = (value+1)/self.count
        self._refresh(f"{self.name}: {int(100*p)}% complete.")
        
    def complete(self):
        self._refresh(f"{self.name}: 100% complete.")
        
    def _display(self, message):
        display(Markdown(message), display_id=str(id(self)))
        
    def _refresh(self, message):
        update_display(Markdown(message), display_id=str(id(self)))

This is a small utility for summarizing warnings, since those tend to take up a lot of space and aren't always something one wants to avoid.
<br>The idea is that one would disable it when looking to debug them.

In [ ]:
from collections import defaultdict
import sys

@contextlib.contextmanager
def tally_warnings():
    with warnings.catch_warnings(record=True) as caught:
        
        yield
        
        tally = defaultdict(int)
        for warning in caught:
            tally[warning.category.__name__] += 1
        
        for category, amount in tally.items():
            print("  ", category + "s:", amount, file=sys.stderr)

## Adjusting MyGaussianNB

The code below is similar to what it was in assignment 1. 

I changed bits of it after reading the solution, I hadn't seen some tricks like the Counter class or using dict.get as a sorting key.

In terms of adapting to missing values, I decided to let it fail gracefully without raising errors as far as possible. This is partially because failing gracefully is a trait of some Python libraries I've grown to like, but mainly because it makes it easier to experiment with later. I can draw graphs with 0 to 100% missing values without worrying that my estimator is going to throw a fit at some arbitrary point I define here.

The fallbacks are as follows:
  - In the training data, each feature ignores nan values.
  - A feature with no valid training values will have invalid parameters.
  - When predicting, features with invalid parameters are ignored. (the likelihood is unaffected by the feature)
  - When predicting, invalid sample feature values are ignored. (the likelihood is unaffected by the feature)
  - If no features affect the likelyhood, the likelihood is one, and only the prior is considered.
    (In other words, the model eventually falls back to predicting the majority class)
  - A nan in the target feature causes the entire record it is in to be ignored.
 
In short, the model uses what features it can, and when there are no features it predicts the majority class.
<br>The one thing it is not design to handle is the case where *none* of the training labels are valid. In that case, it has no class to predict.

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter

class MyGaussianNB(BaseEstimator, ClassifierMixin):          
    
    def fit(self, X, y):
        
        if np.isnan(y).all():
            raise ValueError("No valid training labels given")
        
        self.priors = dict()
        self.means = dict()
        self.vars = dict()
        
        n_records = len(y)
        n_features = X.shape[1]
        classes = Counter(y).items()
        
        for cls, count in classes:
            
            self.priors[cls] = count / n_records
            
            cls_data = X[y == cls]
            self.means[cls] = np.zeros(n_features)
            self.vars[cls] = np.zeros(n_features)
            
            for feature_index in range(n_features):
                
                feature_data = cls_data[:, feature_index]
                
                if np.isnan(feature_data).all():
                    mean = np.nan
                    var = np.nan
                else:
                    mean = np.nanmean(feature_data)
                    var = np.nanvar(feature_data)
                
                self.means[cls][feature_index] = mean
                self.vars[cls][feature_index] = var
                
        return self
    
    def predict(self, X):
        
        predictions = np.zeros(len(X))
        
        for i, sample in enumerate(X):
            
            posteriors = dict()
            
            first = next(iter(self.priors.keys()))
            for cls, prior in self.priors.items():

                likelihood = 1
                
                for feature_index, feature_value in enumerate(sample):
                    
                    if np.isnan(feature_value):
                        continue
                    
                    mean = self.means[cls][feature_index]
                    var = self.vars[cls][feature_index]
                    
                    if np.isnan(mean) or np.isnan(var):
                        continue
                    
                    x = feature_value
                    
                    if var == 0:
                        p = 1 if x == mean else 0

                    else:
                        a = (var * 2 * np.pi)**-0.5
                        b = -(x - mean)**2 / (2 * var)
                        
                        p = a*np.exp(b)

                    likelihood *= p

                posterior = prior * likelihood
                posteriors[cls] = posterior
                
            predictions[i] = max(posteriors, key=posteriors.get)
            
        return predictions

## Data Preparation

Data prep. Some of this is still manual, but I have discovered the joys of pipelines too.

The steps are:

  - read data
  - filter classes and features for testing
  - encoding
  - make some of the data missing ("nanification")
  - imputation (for the sklearn case only)
  - normalization
 
Reading, filtering, and encoding are done below. 

One Hot Encoding is used for the categorical descriptive features.  The target feature is given an ordinal encoding, which isn't needed - plain labels would have been fine. However, it works neatly with certain graphs and also prevents pd.get_dummies from affecting it, so I'm leaving it be.

The ability to exclude features is there for exploration/experimentation - the penguins dataset in particular behaves very differently when the categorical features are removed.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer

def encode(data, target_feature):
    
    encoder = OrdinalEncoder()
    data[[target_feature]] = encoder\
        .fit_transform(data[[target_feature]])
    
    data = pd.get_dummies(data)
    
    return data
  
def read_data(path, target_feature, 
              allowed_classes=None, allowed_features=None,
             verbose=False):
    
    data = pd.read_csv(path, index_col = 0)  
    
    if verbose:
        display(data.head(2))
    
    if allowed_classes is not None:
        allowed_rows  = data[target_feature].isin(allowed_classes)
        data = data[allowed_rows]
        data = data.reset_index(drop=True)
        
    if allowed_features is not None:
        allowed_features = set(allowed_features)
        allowed_features.add(target_feature)
        data = data[allowed_features]
    
    if verbose:
        display(data.head(2))
    
    data = encode(data, target_feature)
    
    # Move the target feature to the end of the dataset.
    # The exact position is unimportant, but having it in a fixed position
    # means that it can be easily identified after the column names are lost.
    
    features = list(data.columns)
    features.remove(target_feature)
    features.insert(0, target_feature)
    data = data[features]
    
    if verbose:
        display(data.head(2))

    raw_data = data.to_numpy()    
    return raw_data

Test/example:

In [ ]:
data = read_data(
    path="penguins_af.csv", 
    target_feature="species", 
    allowed_classes=["Adelie","Chinstrap"],
    allowed_features=["island", "bill_length_mm", "body_mass_g", "year"],
    verbose=True
)

The returned data, displayed below, has not been split into X and y components yet. This is so that an imputation can be applied which acts across the whole dataset at once, such as KNN.

In [ ]:
data

The indexing when splitting it is a little awkward and depends on how the read_data function works, so I've defined  a function with it here.

"Boxing" below refers to having y as a 2D array as opposed to a 1D array. It can be useful to have in 2D for pipelines and transforms, but it needs to be in 1D when actually used as the training set.

In [ ]:
def split(data, box_y=False):
    
    X = data[:,1:]
    
    if box_y:
        y = data[:,0:1]
    else:
        y = data[:,0]
    
    return X, y

def unbox(y):
    
    return y[:,0]

Next, a transform for making random subsets of the data NaN.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from numpy.random import choice

def nanify(X, p):
    
    N = len(X)
    M = int(N*p)
    
    X = X.copy()
    X[choice(N, M, replace=False)] = np.nan
    
    return X

class Nanifier(BaseEstimator, TransformerMixin):
    
    def __init__(self, p=0.5, p0=None):
        
        if p0 is None:
            p0 = p
        
        self.p = p
        self.p0 = p0
        
    def fit(self, X, y=None):
        
        return self
    
    def transform(self, X):
        
        X = X.copy()
        
        for i in range(X.shape[1]):
            p = self.p0 if i == 0 else self.p
            X[:,i] = nanify(X[:,i], p)
        
        return X

An example of the above, displayed with spy plots.

In [ ]:
def spy(X):
    
    X = X.transpose()
    
    plt.figure(figsize=(10,2))
    plt.axis("off")
    plt.spy((~np.isnan(X)).astype(int));
    
data = read_data("penguins_af.csv", "species")
X, y = split(data, box_y=True)

for p in [0.1, 0.5, 0.9]:
    
    nf = Nanifier(p)
    spy(nf.fit_transform(X))
    spy(nf.fit_transform(y))

Finally, it's handy to have the penguins loaded in and named.

In [ ]:
penguins = read_data("penguins_af.csv", "species")

penguins_restricted = read_data("penguins_af.csv", "species",
    allowed_classes = ["Adelie","Chinstrap"],
    allowed_features = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]
)

That is all the data prep utility that is defined in advance - normalization and imputation are done using pipelines.

A simple GNB setup using the above utilities would look like:

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB

data = penguins

combined_pl = make_pipeline(Nanifier(p=0.1), SimpleImputer(strategy="most_frequent"))
data = combined_pl.fit_transform(data, data)

X, y = split(data)

X_pl = make_pipeline(MinMaxScaler(), GaussianNB())
pd.Series(cross_val_score(X_pl, X, y)).mean()

The score is relatively low, even with just 10% of values missing. 

That being said, it only scored ~70% in assignment 1 when all features and classes present. 
<br>If the features and classes are restricted, the model does better:

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.impute import KNNImputer

data = penguins_restricted

combined_pl = make_pipeline(Nanifier(p=0.1), SimpleImputer(strategy="most_frequent"))
data = combined_pl.fit_transform(data, data)

X, y = split(data)

X_pl = make_pipeline(MinMaxScaler(), GaussianNB())
pd.Series(cross_val_score(X_pl, X, y)).mean()

Nothing new there, just sanity checking.

## Comparisons

Here the two strategies of dealing with missing values are compared. Imputation is used with SKLearn. Ignoring missing values can't be done through SKLearn, but the custom MyGNB has been adjusted to allow for it.

These functions represent the experiments to be considered:
  - Ingoring missing values using MyGNB.
  - Univariate imputation, using the mean.
  - Multivariate imputation, using KNN.

In [ ]:
from sklearn.impute import KNNImputer

def ignore_missing(p):
    
    X_pl = make_pipeline(
        Nanifier(p=p), 
        MinMaxScaler(),
        MyGaussianNB()
    )
    
    return X_pl

def simple_imputation(p):
    
    X_pl = make_pipeline(
        Nanifier(p=p), 
        SimpleImputer(strategy="mean"),
        MinMaxScaler(),
        GaussianNB()
    )
    
    return X_pl
    
def knn_imputation(p):
    
    X_pl = make_pipeline(
        Nanifier(p=p), 
        KNNImputer(),
        MinMaxScaler(),
        GaussianNB()
    )
    
    return X_pl

This function scores one or more of the above pipelines for different proportions of missing data. The default (5-fold as of writing this) cross validation is used.

Note that the missing values transform is applied to the data as part of the preprocessing pipeline, and so is applied to both training and test data.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.impute import KNNImputer

def test_with(data, pipelines, samples=5):
    
    np.random.seed(0)
    
    results = defaultdict(dict)
    
    X, y = split(data)
    
    progress = Progress("Scoring", samples*len(pipelines))
    
    for i, pipeline in enumerate(pipelines):
        for j, p in enumerate(np.linspace(0, 1, samples, endpoint=False)):

            score = cross_val_score(pipeline(p), X, y)
            score = pd.Series(score)
            
            results[pipeline.__name__][p] = score

            progress.update(i*samples + j)
            
    progress.complete()
    
    for key, value in results.items():
        results[key] = pd.DataFrame.from_dict(value)
    
    return results

Finally, a function to graph the output of the experiment.

In [ ]:
def graph(results, title):
    
    plt.figure()
    
    plt.title(title)
    plt.xlabel("Proportion Missing")
    plt.ylabel("Cross-Val Score")

    colors = "rbkm"
    for i, (name, result) in enumerate(results.items()):

        line = result.mean()
        style = "." + colors[i]

        line.plot(kind="line", style=style, label=name)

    if len(results) > 1:
        plt.legend()

First, a graph of how the sklearn model works with missing values.

This is partially a sanity check, and partially to see how quickly the accuracy fades.

In [ ]:
results = test_with(penguins, [simple_imputation], samples=200)
graph(results, "SImple Imputation, Penguins")

This is a lovely trend, a little poetic - it implies that a bit of missing data strengthens the model.

This trend is not replicated at all with the restricted penguins data.

In [ ]:
with tally_warnings():
    results = test_with(penguins_restricted, [simple_imputation], samples=200)

graph(results, "SImple Imputation, Penguins (Restricted)")

The runtime warnings above occur very near the end, as there starts to be very little data left. It's not surprising that the numerical calculations break down at that point.

Comparing the three on the restricted penguins dataset:

In [ ]:
experiments = [
    ignore_missing,
    simple_imputation,
    knn_imputation
]

with tally_warnings():
    results = test_with(penguins_restricted, experiments, samples=50)

graph(results, "Penguins (Restricted)")

They all do pretty similar. It seems like KNN has a slight edge over Simple Imputation, and Ignoring does slightl better than KNN. 

I'm not surprised KNN does better than the simple one, since it takes more data into account when imputing. I'm a little surprised at how well ignoring the data is doing, though one explanation there might be that relying and less, but more reliable, data is proving more effective in this case.

Finally, considering other datasets:

In [ ]:
def with_df(path, target_feature, samples=50):
    
    data = read_data(path, target_feature)
    
    with tally_warnings():
        results = test_with(data, experiments, samples)

    graph(results, path)

The diabetes dataset shows a similar trend to the restricted penguins one.

In [ ]:
with_df("diabetes.csv", "neg_pos")

I kept the forecast one only as an example of how chaotic the results from the smaller datasets are - Atheletes and ApplesPears are similar.

All three methods seem to do about the same here.

In [ ]:
with_df("Forecast.csv", "Go-Out")

Glass V2 shows an increase in accuracy up to 50% missing data, before falling back. The accuracy is almost never above 50%. Near the end, ignoring does better than KNN which does better than simple imputation.

In [ ]:
with_df("glassV2.csv", "Type")

For MamMass, despite there being gaps between the models at any given point, I'd argue that the rate of loss in accuracy is more relevant here, and the three seem fairly even in that regard.

Also, the change is quite small, the y-axis goes from 0.7 down to 0.5.

In [ ]:
with_df("MamMass.csv", "Severity", samples=20)

MyGNB does better than the SKLearn one with no missing data, which makes comparing the methods of dealing with missing data unreliable here. It's a good reminder that there are other factors at play.

In [ ]:
with_df("penguins_af.csv", "species")

I don't know what to make of the survival one, the models do not seem to be drawing much information from the data at all.

In [ ]:
with_df("survival.csv", "Class")

The vehicles and wine datasets also show the Ignoring > KNN > Simple trend near for missing proportions > ~60%. Simple imputation starts out stronger in the vehicles set.

In [ ]:
with_df("vehicle.csv", "TYPE")

In [ ]:
with_df("wine.csv", "class")

# Conclusion

The two methods perform very similarly. Which one wins out seems to depend on the dataset. With the ones above, when there is a lot of missing data (~70%), it seems ignoring the data works more often than not better than the others, with KNN second. Simple imputation works well for smaller amounts of missing data, up to around 40%.

Imputing means some of the data will be wrong, but more of the correct data will be used. Ignoring means less of the data will be used, but what will be used is perhaps more reliable. ("wrong" and "correct" and "reliable" really does depend on the dataset) This really seems like one of those many, many, little controls that can be useful to tweak in specific situations, for fine tuning.

It makes sense that SKLearn chose to support imputation and not the tailored fallbacks, given the mostly minor difference in performance. The fallbacks need to be carefully designed and implemented for every model anew, where as imputation is a separate preprocessing concern that can be engineered separately. This is also an interesting reminder that while SKLearn offers a lot in a very easy to access way, it is far from exhaustive and custom implementation and extension has the potential to further improve models.

Finally, I'm still fascinated by the fact that some of the features in the penguins dataset diminish the SKLearn's NB model's ability to learn.